## 1. Налаштування та імпорти

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Додаємо src до шляху
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from mle_star_ablation import (
    AblationConfig,
    build_pipeline,
    generate_ablation_configs,
    DatasetLoader,
    calculate_classification_metrics,
    generate_statistical_report,
    plot_comparison_barplot,
    plot_boxplot,
    pairwise_comparison,
    summarize_statistics
)

# Налаштування відображення
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✓ Імпорти завершені")

## 2. Завантаження даних

Використаємо вбудований датасет breast_cancer для демонстрації.

In [ ]:
# Завантаження даних
DATASET_NAME = 'breast_cancer'
RANDOM_STATE = 42

X_train, X_test, y_train, y_test = DatasetLoader.load_dataset(
    dataset_name=DATASET_NAME,
    random_state=RANDOM_STATE
)

print(f"Датасет: {DATASET_NAME}")
print(f"Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Розподіл класів (train): {np.bincount(y_train)}")

## 3. Одиничний експеримент

Спочатку протестуємо одну конфігурацію.

In [ ]:
# Створення базової конфігурації
config = AblationConfig(
    use_scaling=True,
    use_feature_engineering=False,
    use_hyperparam_tuning=False,
    use_ensembling=False,
    model_type='logistic'
)

print(f"Конфігурація: {config.get_name()}")
print(f"Параметри: {config.to_dict()}")

In [ ]:
# Побудова та тренування пайплайна
pipeline = build_pipeline(config, random_state=RANDOM_STATE)
pipeline.fit(X_train, y_train)

# Передбачення
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)

# Обчислення метрик
metrics = calculate_classification_metrics(y_test, y_pred, y_proba)

print("\nРезультати:")
for metric, value in metrics.items():
    print(f"  {metric}: {value:.4f}")

## 4. Абляційний аналіз

Запускаємо кілька конфігурацій для порівняння впливу різних компонентів.

In [ ]:
# Генерація конфігурацій
configs = generate_ablation_configs()

print(f"Згенеровано {len(configs)} конфігурацій:")
for i, cfg in enumerate(configs, 1):
    print(f"  {i}. {cfg.get_name()}")

In [ ]:
# Запуск експериментів з кількома повторами
N_RUNS = 5
results_dict = {}

for config in configs:
    config_name = config.get_name()
    scores = []
    
    print(f"\nЗапуск: {config_name}")
    
    for run_idx in range(N_RUNS):
        # Різний random_state для кожного повтору
        rs = RANDOM_STATE + run_idx
        
        # Перезавантаження даних
        X_tr, X_te, y_tr, y_te = DatasetLoader.load_dataset(
            DATASET_NAME, random_state=rs
        )
        
        # Тренування та оцінка
        pipe = build_pipeline(config, random_state=rs)
        pipe.fit(X_tr, y_tr)
        
        y_p = pipe.predict(X_te)
        try:
            y_pb = pipe.predict_proba(X_te)
        except:
            y_pb = None
        
        m = calculate_classification_metrics(y_te, y_p, y_pb)
        scores.append(m['accuracy'])
    
    results_dict[config_name] = np.array(scores)
    print(f"  → Accuracy: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

print("\n✓ Експерименти завершені")

## 5. Статистичний аналіз

Перевіряємо статистичну значущість різниці між конфігураціями.

In [ ]:
# Підсумкова статистика
summary_df = summarize_statistics(results_dict)
print("\nПідсумкова статистика:")
print(summary_df.to_string(index=False))

In [ ]:
# Попарні порівняння
comparison_df = pairwise_comparison(results_dict, alpha=0.05, correction='bonferroni')

print("\nПопарні порівняння (з поправкою Бонферроні):")
print(comparison_df[['config_a', 'config_b', 'mean_diff', 'p_value', 'bonferroni_significant']].to_string(index=False))

In [ ]:
# Повний статистичний звіт
report = generate_statistical_report(results_dict, alpha=0.05)
print(report)

## 6. Візуалізація

Створюємо графіки для наочного представлення результатів.

In [ ]:
# Барчарт з довірчими інтервалами
plot_comparison_barplot(results_dict, metric_name='Accuracy', figsize=(14, 6))

In [ ]:
# Boxplot для аналізу розподілів
plot_boxplot(results_dict, metric_name='Accuracy', figsize=(14, 6))

## 7. Висновки

На основі проведеного аналізу:

In [ ]:
# Топ-3 конфігурації
top3 = summary_df.head(3)

print("Топ-3 конфігурації за accuracy:")
print("="*70)
for idx, row in top3.iterrows():
    print(f"{idx+1}. {row['configuration']}")
    print(f"   Mean: {row['mean']:.4f} ± {row['std']:.4f}")
    print(f"   95% CI: [{row['ci_lower']:.4f}, {row['ci_upper']:.4f}]")
    print()

In [ ]:
# Аналіз впливу компонентів
baseline = results_dict.get('logistic', results_dict[list(results_dict.keys())[0]])
baseline_mean = np.mean(baseline)

print("\nВплив компонентів (відносно базової конфігурації):")
print("="*70)
for config_name, scores in results_dict.items():
    improvement = np.mean(scores) - baseline_mean
    improvement_pct = (improvement / baseline_mean) * 100
    print(f"{config_name:40s}: {improvement:+.4f} ({improvement_pct:+.2f}%)")

## 8. Збереження результатів

In [ ]:
# Збереження у CSV
output_dir = Path('../results')
output_dir.mkdir(exist_ok=True)

# Детальні результати
all_results = []
for config_name, scores in results_dict.items():
    for run_idx, score in enumerate(scores):
        all_results.append({
            'configuration': config_name,
            'run': run_idx + 1,
            'accuracy': score
        })

df_results = pd.DataFrame(all_results)
df_results.to_csv(output_dir / 'notebook_results.csv', index=False)
print(f"Результати збережено в {output_dir / 'notebook_results.csv'}")

# Статистика
summary_df.to_csv(output_dir / 'notebook_summary.csv', index=False)
print(f"Статистика збережена в {output_dir / 'notebook_summary.csv'}")

---

**Дипломна робота:** Абляційний аналіз та статистична оцінка важливості компонентів багатостадійного ML-конвеєра

**Студент:** Фефелов Ілля Олександрович

**МАУП, 2025**